<a href="https://colab.research.google.com/github/Lisker2/ML/blob/main/simple_implementation/HMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import scipy
import itertools

In [2]:
state = 3
A = np.array([
    [0.1, 0.4, 0.5],
    [0.3, 0.2, 0.5],
    [0.1, 0.6, 0.3]
])
_, left_eigenvector = scipy.linalg.eig(A, left=True, right=False)
pi = left_eigenvector[:,0] / np.sum(left_eigenvector[:,0])

print('Number of state is\n', state)
print('Transition matrix\n', A)
print('Initial probability distribution\n', pi)

Number of state is
 3
Transition matrix
 [[0.1 0.4 0.5]
 [0.3 0.2 0.5]
 [0.1 0.6 0.3]]
Initial probability distribution
 [0.18055556 0.40277778 0.41666667]


In [3]:
def sequence_generator(num, start):
  res = []
  current = start
  res.append(current)
  for i in range(num):
    current = np.random.choice([i for i in range(state)], p=A[current])
    res.append(current)
  return res

In [4]:
res_100 = sequence_generator(100, 0)
res_500 = sequence_generator(500, 0)
res_100000 = sequence_generator(500, 0)

In [5]:
def eval_res(res):

  n = len(res)
  transition = np.zeros((state, state))
  pi = np.zeros(state)

  for i in range(n - 1):
    current = res[i]
    next = res[i + 1]
    pi[current] += 1
    transition[current][next] += 1
  
  transition = [transition[i]/pi[i] for i in range(state)]
  pi /= n 

  return transition, pi

In [6]:
print('The estimation of using generated 100 sequences:\n',eval_res(res_100))
print('The estimation of using generated 500 sequences:\n',eval_res(res_500))
print('The estimation of using generated 100000 sequences:\n',eval_res(res_100000))

The estimation of using generated 100 sequences:
 ([array([0.11764706, 0.52941176, 0.35294118]), array([0.1627907 , 0.23255814, 0.60465116]), array([0.175, 0.625, 0.2  ])], array([0.16831683, 0.42574257, 0.3960396 ]))
The estimation of using generated 500 sequences:
 ([array([0.15384615, 0.32967033, 0.51648352]), array([0.29569892, 0.1827957 , 0.52150538]), array([0.0941704 , 0.5470852 , 0.35874439])], array([0.18163673, 0.37125749, 0.44510978]))
The estimation of using generated 100000 sequences:
 ([array([0.09302326, 0.3255814 , 0.58139535]), array([0.28358209, 0.23383085, 0.48258706]), array([0.09859155, 0.5915493 , 0.30985915])], array([0.17165669, 0.4011976 , 0.4251497 ]))


The results of generated sequences of lengths 100 and 500 differ slightly from the orginal setup. Because of the length limit, it's just a rough approximation. However, if one increases the length to 10000, the result is almost identical to the orginal setup since the length is long enough to ignore the start state. 

In [7]:
B = np.array([
    [0.5, 0.1, 0.1, 0.1, 0.2],
    [0.1, 0.3, 0.4, 0.1, 0.1],
    [0.2, 0.1, 0.1, 0.3, 0.4]
])
observation = [0, 1, 2, 3, 4]

In [8]:
# Enumerate all the combinations. In this case 3**5 = 243 times calculation.

def dumb(observation):
  n = len(observation)

  all = list(itertools.product([_ for _ in range(state)], repeat=n))
  best_seq = all[0]
  best_p = 0
  sum = 0

  for seq in all:

    temp_p = pi[seq[0]] * B[seq[0]][observation[0]]
    for t in range(1, n):
      temp_p *= (A[seq[t - 1]][seq[t]] * B[seq[t]][observation[t]])

    if temp_p > best_p:
      best_p, best_seq  = temp_p, seq
  
  return best_seq, best_p

In [9]:
dumb(observation)

((2, 1, 1, 2, 2), 2.16e-05)

In [10]:
# This follows pseudo code in ppt.

def viterbi(observation):

  n = len(observation)
  viterbi = np.zeros((state, n))
  backpointer = np.zeros((state, n))
  best_path_prob = 0
  best_path = [0 for _ in range(n)]


  viterbi[:, 0] = pi * B[:, observation[0]]

  for t in range(1, n):
    for s in range(state):

      temp = viterbi[:, t - 1] * A[:, s] * B[s][observation[t]]
      viterbi[s, t] = max(temp)
      backpointer[s, t] = np.argmax(temp)
  
  best_path_prob = max(viterbi[:, -1])
  best_path[-1] = np.argmax(viterbi[:, -1])
  
  for i in range(n - 2, -1, -1):
    best_path[i] = int(backpointer[int(best_path[i + 1]), i + 1])

  return best_path, best_path_prob

In [11]:
viterbi(observation)

([2, 1, 1, 2, 2], 2.1599999999999996e-05)

Looks good.